In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('kddcup.data_10_percent_corrected')

In [3]:
categorical_index = [1,2,3,6,11,20,21]
unique_list = []

unique1 = data.iloc[:,1].unique()
unique_list.append(unique1)
unique2 = data.iloc[:,2].unique()
unique_list.append(unique2)
unique3 = data.iloc[:,3].unique()
unique_list.append(unique3)
unique4 = data.iloc[:,6].unique()
unique_list.append(unique4)
unique5 = data.iloc[:,11].unique()
unique_list.append(unique5)
unique6 = data.iloc[:,20].unique()
unique_list.append(unique6)
unique7 = data.iloc[:,21].unique()
unique_list.append(unique7)

In [4]:
value = 0
for i in range(len(unique1)):
    data.iloc[:,1] = np.where(data.iloc[:,1]==unique1[i], value, data.iloc[:,1])
    value = value + 1

value = 0
for i in range(len(unique2)):
    data.iloc[:,2] = np.where(data.iloc[:,2]==unique2[i], value, data.iloc[:,2])
    value = value + 1

value = 0
for i in range(len(unique3)):
    data.iloc[:,3] = np.where(data.iloc[:,3]==unique3[i], value, data.iloc[:,3])
    value = value + 1

value = 0
for i in range(len(unique4)):
    data.iloc[:,6] = np.where(data.iloc[:,6]==unique4[i], value, data.iloc[:,6])
    value = value + 1

value = 0
for i in range(len(unique5)):
    data.iloc[:,11] = np.where(data.iloc[:,11]==unique5[i], value, data.iloc[:,11])
    value = value + 1

value = 0
for i in range(len(unique6)):
    data.iloc[:,20] = np.where(data.iloc[:,20]==unique6[i], value, data.iloc[:,20])
    value = value + 1

value = 0
for i in range(len(unique7)):
    data.iloc[:,21] = np.where(data.iloc[:,21]==unique7[i], value, data.iloc[:,21])
    value = value + 1
    

In [5]:
label_values = data.iloc[:,41].unique()
label_values

array(['normal.', 'buffer_overflow.', 'loadmodule.', 'perl.', 'neptune.',
       'smurf.', 'guess_passwd.', 'pod.', 'teardrop.', 'portsweep.',
       'ipsweep.', 'land.', 'ftp_write.', 'back.', 'imap.', 'satan.',
       'phf.', 'nmap.', 'multihop.', 'warezmaster.', 'warezclient.',
       'spy.', 'rootkit.'], dtype=object)

In [6]:
data.iloc[:,41] = np.where(data.iloc[:,41] != 'normal.',1,data.iloc[:,41])
data.iloc[:,41] = np.where(data.iloc[:,41] == 'normal.',0,data.iloc[:,41])

data.iloc[:,41].unique()

array([0, 1], dtype=object)

In [7]:
print(data.shape)
Y = data.iloc[:,41]
Y = np.array(Y)
#Y = Y.reshape(Y.shape[0],1)
print(Y.shape)
data.drop(data.columns[41], axis=1, inplace=True)
print(data.shape)

(494020, 42)
(494020,)
(494020, 41)


In [8]:
from sklearn.model_selection import train_test_split  
X_train, X_test, Y_train, Y_test = train_test_split(data, Y, test_size = 0.20) 

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
#Y_train = Y_train.reshape(Y_train.shape[0],1)
Y_test = np.array(Y_test)
#Y_test = Y_test.reshape(Y_test.shape[0],1)

print("X_train : ",X_train.shape)
print("X_test : ",X_test.shape)
print("Y_train : ",Y_train.shape)
print("Y_test : ",Y_test.shape)

X_train :  (395216, 41)
X_test :  (98804, 41)
Y_train :  (395216,)
Y_test :  (98804,)


In [9]:
# Preprocessing in order to do the computation faster doing scaling can use MinMaxScaler as well...
# Check how SVM learns in terms of distance calculation


from sklearn import svm
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder


X_train = X_train.astype(float)
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
print("X_train Shape : ", X_train.shape)

Y_train = np.array(Y_train)
encoder = LabelEncoder()
encoder.fit(Y_train)
Y_train = encoder.transform(Y_train)
print("Y_train Shape : ",Y_train.shape)

X_test = X_test.astype(float)
scaler = preprocessing.StandardScaler()
scaler.fit(X_test)
X_test = scaler.transform(X_test)
print("X_train Shape : ", X_train.shape)

Y_test = encoder.transform(Y_test)
print("Y_test Shape : ",Y_test.shape)


X_train Shape :  (395216, 41)
Y_train Shape :  (395216,)
X_train Shape :  (395216, 41)
Y_test Shape :  (98804,)


In [10]:
from sklearn import svm
import time

svm_model = svm.SVC(kernel='linear',C=1.0)

start = time.time()
svm_model.fit(X_train,Y_train)
end = time.time()

print("Training Time for Binary SVM : ", (end - start))

# Predicting the Results

predicted = svm_model.predict(X_test)

print("Done Predicting..")


predicted = predicted.ravel()
#print(len(predicted))
Y_predicted = np.empty((len(predicted),))
for i in range(len(predicted)):
    if predicted[i] < 0.5:
        Y_predicted[i] = 0
        #Y_predicted[i] = (Y_predicted[i])
    else:
        Y_predicted[i] = 1
        #Y_predicted[i] = math.floor(Y_predicted[i])

Y_predicted = Y_predicted.astype(int)
Y_test_final = np.array(Y_test)
print(Y_predicted.shape)
print(Y_test_final.shape)
print(type(Y_predicted))
print(type(Y_test_final))

print("Y_predicted Unique and its Frequency :")
y_pre,y_pred_counts = np.unique(Y_predicted,return_counts=True)
print(y_pre,y_pred_counts)

print("Y_test Final unique and its Frequency : ")
y_test_final,y_test_counts = np.unique(Y_test_final,return_counts=True)
print(y_test_final,y_test_counts)


tp,fp,tn,fn = 0,0,0,0
for i in range(len(Y_test_final)):
    if Y_test_final[i] == Y_predicted[i] and Y_test_final[i] == 1:
        tp = tp + 1
    elif Y_test_final[i] == Y_predicted[i] and Y_test_final[i] == 0:
        tn = tn + 1
    elif Y_test_final[i] != Y_predicted[i] and Y_test_final[i] == 0:
        fp = fp + 1
    else:
        fn = fn + 1

accuracy = (tp+tn)/(tp+tn+fp+fn)
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1_score = (2*precision*recall)/(precision + recall)
print("tp :",tp)
print("tn :",tn)
print("fp :",fp)
print("fn :",fn)
print("Accuracy : ",accuracy)
print("Precision : ",precision)
print("Recall : ",recall)
print("f1_score : ",f1_score)



Training Time for Binary SVM :  1354.594537973404
Done Predicting..
(98804,)
(98804,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Y_predicted Unique and its Frequency :
[0 1] [19620 79184]
Y_test Final unique and its Frequency : 
[0 1] [19552 79252]
tp : 79084
tn : 19452
fp : 100
fn : 168
Accuracy :  0.9972875592081292
Precision :  0.9987371186098202
Recall :  0.9978801796800081
f1_score :  0.9983084652477971


In [11]:
## RBF Kernel

svm_model = svm.SVC(kernel='rbf',C=1.0)
svm_model.fit(X_train,Y_train)
print("Done Modelling...")

# Predicting the Results

predicted = svm_model.predict(X_test)

print("Done Predicting..")


predicted = predicted.ravel()
#print(len(predicted))
Y_predicted = np.empty((len(predicted),))
for i in range(len(predicted)):
    if predicted[i] < 0.5:
        Y_predicted[i] = 0
        #Y_predicted[i] = (Y_predicted[i])
    else:
        Y_predicted[i] = 1
        #Y_predicted[i] = math.floor(Y_predicted[i])

Y_predicted = Y_predicted.astype(int)
Y_test_final = np.array(Y_test)
print(Y_predicted.shape)
print(Y_test_final.shape)
print(type(Y_predicted))
print(type(Y_test_final))

print("Y_predicted Unique and its Frequency :")
y_pre,y_pred_counts = np.unique(Y_predicted,return_counts=True)
print(y_pre,y_pred_counts)

print("Y_test Final unique and its Frequency : ")
y_test_final,y_test_counts = np.unique(Y_test_final,return_counts=True)
print(y_test_final,y_test_counts)


tp,fp,tn,fn = 0,0,0,0
for i in range(len(Y_test_final)):
    if Y_test_final[i] == Y_predicted[i] and Y_test_final[i] == 1:
        tp = tp + 1
    elif Y_test_final[i] == Y_predicted[i] and Y_test_final[i] == 0:
        tn = tn + 1
    elif Y_test_final[i] != Y_predicted[i] and Y_test_final[i] == 0:
        fp = fp + 1
    else:
        fn = fn + 1

accuracy = (tp+tn)/(tp+tn+fp+fn)
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1_score = (2*precision*recall)/(precision + recall)
print("tp :",tp)
print("tn :",tn)
print("fp :",fp)
print("fn :",fn)
print("Accuracy : ",accuracy)
print("Precision : ",precision)
print("Recall : ",recall)
print("f1_score : ",f1_score)


/home/rajat/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Done Modelling...
Done Predicting..
(98804,)
(98804,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Y_predicted Unique and its Frequency :
[0 1] [19583 79221]
Y_test Final unique and its Frequency : 
[0 1] [19552 79252]
tp : 79188
tn : 19519
fp : 33
fn : 64
Accuracy :  0.9990182583701065
Precision :  0.9995834437838452
Recall :  0.9991924494019079
f1_score :  0.9993879083503183
